<a href="https://colab.research.google.com/github/burakemretetik/dl_with_py/blob/main/deep_rnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ==========================================
# BLOCK 1: DATA LOADING
# ==========================================
import os
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

# 1. INSTALL & CONFIGURE
# ------------------------------------------
# We install datasets but immediately KILL the interactive widgets
# to prevent the "Invalid Notebook" corruption error.
try:
    import datasets
except ImportError:
    print("📦 Installing datasets library...")
    os.system('pip install datasets -q')
    import datasets

from datasets import load_dataset
from datasets.utils.logging import set_verbosity_error, disable_progress_bar

# SAFETY SWITCHES
set_verbosity_error()      # Hides "HF_TOKEN" warnings
disable_progress_bar()     # Hides the download bar that breaks Colab

# Device Config
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"🚀 Hardware Selected: {device}")

# 2. LOAD REAL DATA
# ------------------------------------------
print("\n🍳 Downloading Real Recipes (Safe Mode)...")
# We take 5,000 recipes. (Increase to 10,000 later if you want a smarter chef)
try:
    dataset = load_dataset("corbt/all-recipes", split="train[:5000]")
except Exception as e:
    print(f"⚠️ Primary load failed: {e}")
    # Fallback to a smaller slice if the main one times out
    dataset = load_dataset("corbt/all-recipes", split="train[:1000]")

def format_recipe(example):
    # Extract Real Data & Clean it
    ingr = str(example.get('input', '')).replace("['", "").replace("']", "").replace("', '", ", ")
    instr = str(example.get('target', ''))
    return f"<START> \n <INGR> {ingr} \n <INSTR> {instr} <END>"

# Build the text corpus
corpus_list = [format_recipe(x) for x in dataset]
corpus_text = "\n".join(corpus_list)

# 3. TOKENIZATION & MAPPING
# ------------------------------------------
chars = sorted(list(set(corpus_text)))
vocab_size = len(chars)

# Create the Mappings (Crucial for the Model Block)
char_to_idx = {ch: i for i, ch in enumerate(chars)}
idx_to_char = {i: ch for i, ch in enumerate(chars)}

print(f"✅ REAL DATA LOADED SUCCESSFULLY.")
print(f"   - Recipes: {len(corpus_list)}")
print(f"   - Unique Characters: {vocab_size}")
print(f"   - Sample:\n{corpus_list[0][:150]}...")

# 4. DATASET & LOADER
# ------------------------------------------
class RecipeDataset(Dataset):
    def __init__(self, text, seq_length=128):
        self.data = torch.tensor([char_to_idx[c] for c in text], dtype=torch.long)
        self.seq_length = seq_length

    def __len__(self):
        # We stride by seq_length to maximize data usage
        return (len(self.data) - self.seq_length) // self.seq_length

    def __getitem__(self, idx):
        start = idx * self.seq_length
        chunk = self.data[start : start + self.seq_length + 1]
        # Input (x) is chars 0..99, Target (y) is chars 1..100
        return chunk[:-1], chunk[1:]

BATCH_SIZE = 96
train_dataset = RecipeDataset(corpus_text, seq_length=128)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)

🚀 Hardware Selected: cuda

🍳 Downloading Real Recipes (Safe Mode)...
✅ REAL DATA LOADED SUCCESSFULLY.
   - Recipes: 5000
   - Unique Characters: 90
   - Sample:
<START> 
 <INGR> No-Bake Nut Cookies

Ingredients:
- 1 c. firmly packed brown sugar
- 1/2 c. evaporated milk
- 1/2 tsp. vanilla
- 1/2 c. broken nuts (...


In [2]:
# ==========================================
# BLOCK 2: ARCHITECTURE & SAMPLING
# ==========================================
import torch
import torch.nn as nn
import torch.nn.functional as F

class DeepRecipeGRU(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_layers):
        super().__init__()
        self.num_layers = num_layers
        self.hidden_dim = hidden_dim
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.gru = nn.GRU(embed_dim, hidden_dim, num_layers=num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, vocab_size)

    def forward(self, x, hidden):
        embed = self.embedding(x)
        out, hidden = self.gru(embed, hidden)
        out = out.reshape(-1, self.hidden_dim)
        out = self.fc(out)
        return out, hidden

    def init_hidden(self, batch_size, device):
        return torch.zeros(self.num_layers, batch_size, self.hidden_dim).to(device)

    # --- ADVANCED GENERATION ---
    def generate(self, start_str, char_to_idx, idx_to_char, device,
                 length=200, temperature=0.8, top_k=5):
        self.eval()
        input_idxs = [char_to_idx.get(c, 0) for c in start_str]
        input_tensor = torch.tensor(input_idxs, dtype=torch.long).unsqueeze(0).to(device)
        hidden = self.init_hidden(1, device)

        generated_text = start_str

        with torch.no_grad():
            for _ in range(length):
                output, hidden = self(input_tensor, hidden)

                # 1. Apply Temperature (Higher = Weirder, Lower = Safer)
                logits = output[-1] / temperature

                # 2. Top-K Filtering (The "Sanity Check")
                # We zero out probabilities that are NOT in the top K
                top_logits, top_indices = torch.topk(logits, top_k)
                probs = F.softmax(top_logits, dim=0)

                # 3. Sample from the filtered list
                next_token_index = torch.multinomial(probs, 1).item()

                # Map back to the original character index
                real_char_index = top_indices[next_token_index].item()
                next_char = idx_to_char[real_char_index]

                generated_text += next_char

                # Stop if we hit the end tag (Engineering Logic)
                if next_char == '>' and generated_text.endswith('<END>'):
                    break

                input_tensor = torch.tensor([[real_char_index]], dtype=torch.long).to(device)

        return generated_text

In [3]:
# ==========================================
# BLOCK 3: TRAINING WITH CHECKPOINTS
# ==========================================
import os

# Hyperparameters
EMBED_DIM = 256
HIDDEN_DIM = 1024 # Increased for "Deep" capacity
NUM_LAYERS = 3
NUM_EPOCHS = 20  # Train longer for better recipes
LR = 0.002

# Initialize (Assuming you ran the previous data loading block)
model = DeepRecipeGRU(vocab_size, EMBED_DIM, HIDDEN_DIM, NUM_LAYERS).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=LR)
criterion = nn.CrossEntropyLoss()
scaler = torch.cuda.amp.GradScaler()

print(f"🍳 Chef is ready. Params: {sum(p.numel() for p in model.parameters())}")

history = []

for epoch in range(NUM_EPOCHS):
    model.train()
    hidden = model.init_hidden(BATCH_SIZE, device)
    total_loss = 0

    for i, (x, y) in enumerate(train_loader):
        x, y = x.to(device), y.to(device)
        hidden = hidden.detach()
        optimizer.zero_grad()

        with torch.cuda.amp.autocast():
            output, hidden = model(x, hidden)
            loss = criterion(output, y.view(-1))

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)
    history.append(avg_loss)
    print(f"Epoch {epoch+1}/{NUM_EPOCHS} | Loss: {avg_loss:.4f}")

    # --- CHECKPOINT SAVING ---
    # We save the model state so you can download it later
    if (epoch+1) % 5 == 0:
        save_path = f"chef_model_ep{epoch+1}.pth"
        torch.save(model.state_dict(), save_path)
        print(f"💾 Saved checkpoint: {save_path}")

    # Sanity Check Generation
    if (epoch+1) % 2 == 0:
        print("Sample:", model.generate("<START> \n <INGR> chicken", char_to_idx, idx_to_char, device, top_k=3))
        print("-" * 30)

print("🔥 Training Complete!")

/tmp/ipython-input-349155426.py:17: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()
/tmp/ipython-input-349155426.py:33: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():


🍳 Chef is ready. Params: 16648794
Epoch 1/20 | Loss: 2.4663
Epoch 2/20 | Loss: 0.9865
Sample: <START> 
 <INGR> chicken Casserole, Cookies, strawberry saowy, onion, cooked
- 1 c. chopped oniang powder
- 1 tsp. salt and pepper
- 1 c. milk
- 1/2 c. cake mix
- 1/2 c. milk
- 2 Tbsp. salt
- 1/2 tsp. soda
- 1/2 c. milk
- 1/
------------------------------
Epoch 3/20 | Loss: 0.7979
Epoch 4/20 | Loss: 0.7421
Sample: <START> 
 <INGR> chicken breasts
- 2 c. sugar
- 2 c. brown sugar
- 1 Tbsp. chopped onion
- 1 can cream of mushroom soup
- 1/4 c. miniature marshmallows
- 1 c. chopped pecans

Directions:
- Cook carrots and cook macaroni in a
------------------------------
Epoch 5/20 | Loss: 0.7121
💾 Saved checkpoint: chef_model_ep5.pth
Epoch 6/20 | Loss: 0.6932
Sample: <START> 
 <INGR> chicken Soup

Ingredients:
- 2 1/4 c. shredded Cheddar cheese
- 3/4 c. chopped onion
- 1 c. shredded Parmesan cheese (about 1/2 c.)
- 2 c. milk
- 1 1/2 tsp. salt
- 1/2 tsp. soda
- 1/4 tsp. baking soda
- 2 Tb
----------

In [4]:
# ==========================================
# BLOCK 4: THE INTERACTIVE KITCHEN
# ==========================================
def order_recipe(ingredients, temperature=0.8):
    """
    Generates a recipe based on input ingredients.
    - temperature: 0.5 (Safe/Boring) to 1.0 (Creative/Crazy)
    """
    start_str = f"<START> \n <INGR> {ingredients}"
    print(f"👨‍🍳 Chef is thinking about: {ingredients}...\n")

    # Generate 500 characters of recipe
    recipe = model.generate(start_str, char_to_idx, idx_to_char, device,
                            length=600, temperature=temperature, top_k=5)

    # Clean up the output tags for display
    clean_recipe = recipe.replace("<START>", "").replace("<INGR>", "\n📝 INGREDIENTS:").replace("<INSTR>", "\n🍳 INSTRUCTIONS:").replace("<END>", "")
    print(clean_recipe)
    print("\n-------------------------------------------")

In [6]:
# Predict
order_recipe("chicken, garlic, butter", temperature=0.8)

👨‍🍳 Chef is thinking about: chicken, garlic, butter...

 
 
📝 INGREDIENTS: chicken, garlic, butter, salt and pepper.
- Add salt onstant to blend all other ingredients.
- Pour into baking dish.
- Place in greased casserole dish.
- Top with ser of water and pour over chicken.
- Cook until shortening.
- Add sour cream to set, toss to cook until blended.
- Mix well.
- Pour in bag of cooled light and place in warm water for 10 minutes on ungreased cookie sheet.
- Bake at 350° for 50 minutes, then add the flour until the meat is tender.
- To cool. This may be stiff pops
- 1 lb. butter
- 1 c. milk
- 1 tsp. voder
- 2 Tbsp. soda
- 1 tsp. baking powder
- 1/2 c. brown sugar
- 1 tsp. vanilla
- 1 c. ch

-------------------------------------------
